# Final Project: Restaurant Recommander System

## Imports

In [32]:
import pandas as pd
import numpy as np
import yaml
import os
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import svds
from sklearn.metrics import f1_score

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
from pyspark.sql.functions import concat
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [3]:
config_file = open("config.yml", "r")
config = yaml.load(config_file, Loader=yaml.FullLoader)
DATA_DIRECTORY = config["DATA_DIRECTORY"]

In [4]:
spark = SparkSession.builder.appName("ALSMatrixFactorisation").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/05/17 09:06:01 WARN Utils: Your hostname, DESKTOP-2FVJM3V resolves to a loopback address: 127.0.1.1; using 192.168.74.137 instead (on interface eth0)
23/05/17 09:06:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/17 09:06:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Chargement des données

In [5]:
spark_order_df = spark.read.option("header", True).csv(
    os.path.join(DATA_DIRECTORY, "orders.csv")
)
spark_order_df.show()

23/05/17 09:06:17 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------+-----------+----------+-----------+------------+----------+----------------------+------------------------------+-----------+--------+-------------+-------------+----------------+---------------+-------------------+-------------------+--------------------+---------------------+--------------+--------------+-------------------+---------+-------------------+---------------+-------------+----------------------+
|akeed_order_id|customer_id|item_count|grand_total|payment_mode|promo_code|vendor_discount_amount|promo_code_discount_percentage|is_favorite|is_rated|vendor_rating|driver_rating|deliverydistance|preparationtime|      delivery_time|order_accepted_time|driver_accepted_time|ready_for_pickup_time|picked_up_time|delivered_time|      delivery_date|vendor_id|         created_at|LOCATION_NUMBER|LOCATION_TYPE|CID X LOC_NUM X VENDOR|
+--------------+-----------+----------+-----------+------------+----------+----------------------+------------------------------+-----------+---

In [43]:
CUSTOMER_KEY = "CID x LOC_NUM"
df_ratings = spark_order_df.withColumn(
    "CID x LOC_NUM",
    concat(
        spark_order_df["customer_id"], lit(" X "), spark_order_df["LOCATION_NUMBER"]
    ),
)
df_ratings = df_ratings.select(
    "customer_id", "vendor_id", "vendor_rating", "LOCATION_NUMBER", "CID x LOC_NUM"
).where(col("is_rated") == "Yes")

df_ratings.show()

+-----------+---------+-------------+---------------+-------------+
|customer_id|vendor_id|vendor_rating|LOCATION_NUMBER|CID x LOC_NUM|
+-----------+---------+-------------+---------------+-------------+
|    OH64IO0|      310|          5.0|              0|  OH64IO0 X 0|
|    FCPLE31|      157|          5.0|              0|  FCPLE31 X 0|
|    WB681BO|      271|          4.0|              0|  WB681BO X 0|
|    FS229TW|      157|          4.0|              0|  FS229TW X 0|
|    3P9113W|       85|          5.0|              0|  3P9113W X 0|
|    WB681BO|       90|          5.0|              0|  WB681BO X 0|
|    AA31G37|       85|          5.0|              0|  AA31G37 X 0|
|    E4ZWBIY|       83|          5.0|              0|  E4ZWBIY X 0|
|    Y8OGZS1|      196|          2.0|              0|  Y8OGZS1 X 0|
|    I3DAUFL|      300|          1.0|              0|  I3DAUFL X 0|
|    8BZR1IV|       33|          5.0|              0|  8BZR1IV X 0|
|    5O4E3Z3|       92|          4.0|           

In [44]:
df_ratings.count()

20109

In [45]:
indexer = [
    StringIndexer(inputCol=column, outputCol=column + "_index")
    for column in list(set(df_ratings.columns) - set(["vendor_rating"]))
]

pipeline = Pipeline(stages=indexer)
transformed = pipeline.fit(df_ratings).transform(df_ratings)
transformed = transformed.withColumn(
    "vendor_rating", transformed["vendor_rating"].cast("float")
)
transformed = transformed.withColumn(
    "vendor_id_index", transformed["vendor_id_index"].cast("float")
)
transformed.show(5)

+-----------+---------+-------------+---------------+-------------+---------------+-----------------+---------------------+-------------------+
|customer_id|vendor_id|vendor_rating|LOCATION_NUMBER|CID x LOC_NUM|vendor_id_index|customer_id_index|LOCATION_NUMBER_index|CID x LOC_NUM_index|
+-----------+---------+-------------+---------------+-------------+---------------+-----------------+---------------------+-------------------+
|    OH64IO0|      310|          5.0|              0|  OH64IO0 X 0|           30.0|           6380.0|                  0.0|             7444.0|
|    FCPLE31|      157|          5.0|              0|  FCPLE31 X 0|           31.0|           5302.0|                  0.0|             5957.0|
|    WB681BO|      271|          4.0|              0|  WB681BO X 0|           71.0|           3354.0|                  0.0|             3331.0|
|    FS229TW|      157|          4.0|              0|  FS229TW X 0|           31.0|           2695.0|                  0.0|             

In [46]:
(training, test) = transformed.randomSplit([0.8, 0.2], seed=16)

In [50]:
customer_index_key = f"{CUSTOMER_KEY}_index"
als = ALS(
    maxIter=7,
    regParam=0.09,
    rank=25,
    userCol=customer_index_key,
    itemCol="vendor_id_index",
    ratingCol="vendor_rating",
    coldStartStrategy="drop",
    nonnegative=True,
)

model = als.fit(training)

In [51]:
evaluator = RegressionEvaluator(
    metricName="rmse", labelCol="vendor_rating", predictionCol="prediction"
)

predictions = model.transform(test)
rmse = evaluator.evaluate(predictions)

print("RMSE=" + str(rmse))
predictions.orderBy(col("prediction").desc()).show()

RMSE=1.3412027478623807
+-----------+---------+-------------+---------------+-------------+---------------+-----------------+---------------------+-------------------+----------+
|customer_id|vendor_id|vendor_rating|LOCATION_NUMBER|CID x LOC_NUM|vendor_id_index|customer_id_index|LOCATION_NUMBER_index|CID x LOC_NUM_index|prediction|
+-----------+---------+-------------+---------------+-------------+---------------+-----------------+---------------------+-------------------+----------+
|    MLY7UJE|      193|          5.0|              1|  MLY7UJE X 1|           89.0|            103.0|                  1.0|              122.0|  6.434285|
|    6POHNJ1|      295|          4.0|              1|  6POHNJ1 X 1|           61.0|           1529.0|                  1.0|             1377.0| 6.3890758|
|    6POHNJ1|      295|          5.0|              1|  6POHNJ1 X 1|           61.0|           1529.0|                  1.0|             1377.0| 6.3890758|
|    1PQ0SRF|       67|          5.0|         

In [52]:
recs = model.recommendForAllUsers(20).toPandas()
recs

,CID x LOC_NUM_index,recommendations
0,26,"[(98, 4.823955535888672), (15, 4.3828282356262..."
1,27,"[(86, 5.436115741729736), (81, 5.2077317237854..."
2,28,"[(67, 6.94057035446167), (95, 6.86001157760620..."
3,31,"[(89, 5.960110187530518), (77, 5.5547375679016..."
4,34,"[(67, 5.639974594116211), (77, 5.5471220016479..."
...,...,...
8067,9244,"[(46, 1.963230013847351), (77, 1.9309395551681..."
8068,9250,"[(59, 4.9300312995910645), (67, 4.463428497314..."
8069,9261,"[(77, 6.538561820983887), (95, 6.4643774032592..."
8070,9266,"[(95, 5.802989482879639), (77, 5.4607205390930..."


In [53]:
df_recs = (
    recs.recommendations.apply(pd.Series)
    .merge(recs, right_index=True, left_index=True)
    .drop(["recommendations"], axis=1)
    .melt(id_vars=[customer_index_key], value_name="recommendation")
    .drop("variable", axis=1)
    .dropna()
)
df_recs

,CID x LOC_NUM_index,recommendation
0,26,"(98, 4.823955535888672)"
1,27,"(86, 5.436115741729736)"
2,28,"(67, 6.94057035446167)"
3,31,"(89, 5.960110187530518)"
4,34,"(67, 5.639974594116211)"
...,...,...
161435,9244,"(15, 1.3813546895980835)"
161436,9250,"(66, 3.145433187484741)"
161437,9261,"(38, 5.234267234802246)"
161438,9266,"(18, 4.114292144775391)"


In [54]:
df_recs = df_recs.sort_values(customer_index_key)
df_recs = pd.concat(
    [df_recs["recommendation"].apply(pd.Series), df_recs[customer_index_key]], axis=1
)

In [55]:
df_recs.columns = [
    "vendor_id_index",
    "rating",
    customer_index_key,
]
tmp = transformed.select(
    transformed[CUSTOMER_KEY],
    transformed[customer_index_key],
    transformed["vendor_id"],
    transformed["vendor_id_index"],
)
tmp = tmp.toPandas()

In [56]:
dict1 = dict(zip(tmp[customer_index_key], tmp[CUSTOMER_KEY]))
dict2 = dict(zip(tmp["vendor_id_index"], tmp["vendor_id"]))
df_recs_copy = df_recs.copy()
df_recs_copy.loc[:, CUSTOMER_KEY] = df_recs[customer_index_key].map(dict1)
df_recs_copy.loc[:, "vendor_id"] = df_recs["vendor_id_index"].map(dict2)
df_recs_copy = df_recs_copy.sort_values(CUSTOMER_KEY)
df_recs_copy.reset_index(drop=True, inplace=True)

In [58]:
new = df_recs_copy[[CUSTOMER_KEY, "vendor_id", "rating"]]
new["recommendations"] = list(zip(new.vendor_id, new.rating))

res = new[[CUSTOMER_KEY, "recommendations"]]
recommandation_df = (
    res["recommendations"].groupby([res[CUSTOMER_KEY]]).apply(list).reset_index()
)
print(recommandation_df)

/tmp/ipykernel_431/1335006208.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new["recommendations"] = list(zip(new.vendor_id, new.rating))


     CID x LOC_NUM                                    recommendations
0      00HWUU3 X 1  [(188, 4.143406391143799), (43, 3.919334650039...
1      00OT8JX X 3  [(82, 4.836171627044678), (23, 4.9686641693115...
2      00OT8JX X 4  [(250, 5.496040344238281), (115, 5.50965833663...
3      010DVV3 X 0  [(195, 4.004382610321045), (115, 4.52180814743...
4      018GT0H X 2  [(310, 4.796288013458252), (259, 4.31573295593...
...            ...                                                ...
8067   ZZJSJYC X 0  [(85, 5.077360153198242), (298, 4.979196071624...
8068   ZZJX0AR X 0  [(274, 3.4296140670776367), (20, 3.42652916908...
8069   ZZP5BHU X 0  [(271, 3.294734001159668), (298, 3.17901468276...
8070   ZZRJABJ X 2  [(295, 3.7094619274139404), (191, 4.9173674583...
8071   ZZY3N0D X 1  [(115, 7.167243480682373), (221, 6.83455514907...

[8072 rows x 2 columns]


In [59]:
recommandation_df[CUSTOMER_KEY] = recommandation_df[CUSTOMER_KEY].astype(str)
recommandation_df = recommandation_df.explode("recommendations")
recommandation_df["recommanded_vendor"] = recommandation_df["recommendations"].apply(
    lambda x: x[0]
)
recommandation_df["rating"] = recommandation_df["recommendations"].apply(lambda x: x[1])
recommandation_df = recommandation_df.drop(["recommendations"], axis=1)

In [60]:
minimal_rate_for_recommandation = 3.5
recommandation_df["rating"] = recommandation_df["rating"].apply(
    lambda x: 5 if x >= 5 else x
)

In [63]:
recommandation_df.sort_values(by="rating", ascending=False).head()

,CID x LOC_NUM,recommanded_vendor,rating
8071,ZZY3N0D X 1,303,5.0
1347,5XQRWHD X 1,193,5.0
4407,JRLB643 X 1,265,5.0
1348,5XQRWHD X 2,193,5.0
4406,JRLB643 X 0,149,5.0


In [64]:
recommandation_df.describe()

,rating
count,161440.000000
mean,3.901870
std,1.131642
min,0.000000
25%,3.359153
50%,4.158417
75%,4.904993
max,5.000000


In [65]:
recommandation_df.to_csv(os.path.join(DATA_DIRECTORY, "recommandation.csv"))

In [ ]:
# Données relatives aux commandes des customers aux vendors
order_df = pd.read_csv(os.path.join(DATA_DIRECTORY, "orders.csv"))

# Données des vendeurs(localisation, identifiant)
vendors_df = pd.read_csv(os.path.join(DATA_DIRECTORY, "vendors.csv"))

# Données des clients (date de naissance, ID, genre)
train_customer_df = pd.read_csv(os.path.join(DATA_DIRECTORY, "train_customers.csv"))

# Localisations des clients
train_locations_df = pd.read_csv(os.path.join(DATA_DIRECTORY, "train_locations.csv"))

# Ensemble de tests sur les données des clients
test_customer_df = pd.read_csv(os.path.join(DATA_DIRECTORY, "test_customers.csv"))

# Ensemble de tests sur les localisations des clients
test_locations_df = pd.read_csv(os.path.join(DATA_DIRECTORY, "test_locations.csv"))

/tmp/ipykernel_361/3856822910.py:2: DtypeWarning: Columns (15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  order_df = pd.read_csv(os.path.join(DATA_DIRECTORY, "orders.csv"))


KMeans Clustering

In [ ]:
train_locations_df = train_locations_df.dropna()

In [ ]:
from sklearn.cluster import KMeans

# Charger votre dataframe avec les colonnes latitude et longitude (supposons que le dataframe s'appelle 'df')
# df = ...

# Sélectionner les colonnes latitude et longitude

data = train_locations_df[["latitude", "longitude"]].copy()

# Spécifier le nombre de clusters que vous souhaitez obtenir
k = 6

# Créer une instance de l'algorithme K-Means
kmeans = KMeans(n_clusters=k)

# Appliquer l'algorithme aux données
kmeans.fit(data)

# Obtenir les étiquettes de cluster pour chaque point
labels = kmeans.labels_

# Ajouter les étiquettes de cluster à votre dataframe
train_locations_df["cluster_label"] = labels.copy()

/home/bastien/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
train_locations_df.groupby("cluster_label").count()

,customer_id,location_number,location_type,latitude,longitude
cluster_label,,,,,
0,12847,12847,12847,12847,12847
1,18877,18877,18877,18877,18877
2,403,403,403,403,403
3,9,9,9,9,9
4,76,76,76,76,76
5,80,80,80,80,80


In [ ]:
test_customer_df.columns

Index(['akeed_customer_id', 'gender', 'dob', 'status', 'verified', 'language',
       'created_at', 'updated_at'],
      dtype='object')

In [ ]:
order_with_loc_df = pd.merge(
    order_df[
        ["customer_id", "LOCATION_NUMBER", "vendor_id", "vendor_rating", "is_rated"]
    ],
    train_locations_df,
    left_on=["customer_id", "LOCATION_NUMBER"],
    right_on=["customer_id", "location_number"],
)
order_with_loc_df["is_rated"] = order_with_loc_df["is_rated"].apply(
    lambda b: 1 if b == "Yes" else 0
)
order_with_loc_df["is_rated"] = order_with_loc_df["is_rated"].fillna(0)
order_with_loc_df["vendor_rating"] = order_with_loc_df["vendor_rating"].fillna(0)
order_with_loc_df

,customer_id,LOCATION_NUMBER,vendor_id,vendor_rating,is_rated,location_number,location_type,latitude,longitude
0,92PEE24,0,105,0.0,0,0,NaN,-0.132100,-78.575297
1,92PEE24,0,105,0.0,0,0,NaN,-0.132100,-78.575297
2,92PEE24,0,105,0.0,0,0,NaN,-0.132100,-78.575297
3,QS68UD8,0,294,0.0,0,0,Work,-0.393396,-78.544417
4,MB7VY5F,0,83,0.0,0,0,NaN,-1.072823,-78.464121
...,...,...,...,...,...,...,...,...,...
135298,3S6VG6R,1,199,5.0,1,1,NaN,2.284875,0.717124
135299,ND4PIJL,0,907,0.0,0,0,NaN,13.380083,-1.387421
135300,1NRK5HF,0,105,0.0,0,0,NaN,-0.772600,0.231851
135301,QDXLWM7,1,28,0.0,0,1,NaN,1.751487,0.375234


In [ ]:
train_order_df, test_order_df = train_test_split(
    order_with_loc_df, test_size=0.2, random_state=42
)
train_order_df, val_order_df = train_test_split(
    train_order_df, test_size=0.2, random_state=42
)
train_order_df

,customer_id,LOCATION_NUMBER,vendor_id,vendor_rating,is_rated,location_number,location_type,latitude,longitude
121366,5ARTXD7,1,681,5.0,1,1,NaN,-0.036682,-0.069342
63263,JTDL6KB,0,386,0.0,0,0,NaN,-0.492942,0.179312
30695,H9REAWE,0,78,0.0,0,0,Home,-0.025458,-78.587900
80592,7I7SY2D,1,4,0.0,0,1,NaN,0.415142,0.931376
132382,EIA3K80,3,92,0.0,0,3,Other,0.107888,0.596460
...,...,...,...,...,...,...,...,...,...
25191,4QNVW0N,1,176,0.0,0,1,Other,0.141303,-78.607609
35276,NBHJZZB,0,78,0.0,0,0,NaN,0.275762,-78.623499
124517,3I2NFF0,0,289,0.0,0,0,NaN,0.431060,-0.005375
51877,0Z52KAY,1,843,0.0,0,1,NaN,-0.795356,0.082256


In [ ]:
VAL_KEY = "val"
PRED_KEY = "pred"
VALIDATION_KEY = "validation"
TRAIN_KEY = "training"

In [ ]:
def generate_comparison_df(train_set, val_set):
    train_set = train_set.rename(columns={PRED_KEY: TRAIN_KEY})
    val_set = val_set.rename(columns={PRED_KEY: VALIDATION_KEY})
    comparison_df = pd.merge(train_set, val_set, on=VAL_KEY)
    return comparison_df


def calculate_f1_score(train_set, val_set):
    df = generate_comparison_df(train_set, val_set)
    true_labels = df[VALIDATION_KEY]
    predicted_labels = df[TRAIN_KEY]

    f1 = f1_score(true_labels, predicted_labels)

    return f1


calculate_f1_score(train_set, val_set)

NameError: name 'train_set' is not defined